In [35]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 

In [36]:
tf.__version__

'2.19.0'

In [37]:
pf=pd.read_csv("AmericanExpressSheet.csv")

In [38]:
x= pf.iloc[:,0:-1].values
y= pf.iloc[:,-1].values

In [39]:
print(x)

[[553 'Delhi' 'Female' ... 4 1 274150]
 [447 'Bengaluru' 'Male' ... 4 1 519360]
 [501 'Delhi' 'Female' ... 4 1 545501]
 ...
 [627 'Mumbai' 'Female' ... 4 0 494067]
 [600 'Bengaluru' 'Female' ... 2 1 109375]
 [553 'Delhi' 'Male' ... 4 1 180031]]


In [40]:
print(y)

[0 0 0 ... 0 1 0]


In [41]:
#To find the NaN values
pf.isnull().values.any()
pf[pf.isnull().any(axis=1)]

,Credit Score,Geography,Gender,Age,Customer Since,Current Account,Num of products,UPI Enabled,Estimated Yearly Income,Closed


Gender Column Encoding


In [42]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
x[:,2]= label_encoder.fit_transform(x[:,2])

In [43]:
print(x)

[[553 'Delhi' 0 ... 4 1 274150]
 [447 'Bengaluru' 1 ... 4 1 519360]
 [501 'Delhi' 0 ... 4 1 545501]
 ...
 [627 'Mumbai' 0 ... 4 0 494067]
 [600 'Bengaluru' 0 ... 2 1 109375]
 [553 'Delhi' 1 ... 4 1 180031]]


In [44]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x=np.array(ct.fit_transform(x))

In [45]:
print(x)

[[0.0 1.0 0.0 ... 4 1 274150]
 [1.0 0.0 0.0 ... 4 1 519360]
 [0.0 1.0 0.0 ... 4 1 545501]
 ...
 [0.0 0.0 1.0 ... 4 0 494067]
 [1.0 0.0 0.0 ... 2 1 109375]
 [0.0 1.0 0.0 ... 4 1 180031]]


In [46]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [47]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


ANN


Initialization

In [48]:
ann= tf.keras.models.Sequential()

Adding input layer and first hidden layer

In [49]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding second hidden layer


In [50]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding output layer

In [51]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ANN Training

Compiling ANN

In [53]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Training on training dataset

In [54]:
ann.fit(x_train,y_train,batch_size=32,epochs=120)

Epoch 1/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7460 - loss: 0.5868
Epoch 2/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8003 - loss: 0.4659
Epoch 3/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8025 - loss: 0.4506
Epoch 4/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7907 - loss: 0.4531
Epoch 5/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7970 - loss: 0.4384
Epoch 6/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8006 - loss: 0.4367
Epoch 7/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8069 - loss: 0.4260
Epoch 8/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8080 - loss: 0.4237
Epoch 9/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7991 - loss: 0.4348
Epoch 10/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8126 - loss: 0.4273
Epoch 11/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8138 - loss: 0.4234
Epoch 12/120
249/249 ━━━━━━━━━━━━━━━━━━━━

Predictions

Single predictions

In [55]:

print(ann.predict(sc.transform([[0.0 , 1.0, 0.0, 501, 0, 32, 2, 0.0, 4, 1, 545501]]))>0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
[[False]]


In [56]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step
[[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [0 0]
 [1 1]]


Confusion matrix

In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1526   47]
 [ 253  160]]


0.8489425981873112